In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.image import resize
from tensorflow.keras.applications import VGG16, ResNet50

train_df = pd.read_csv('C:/Users/nagen/Downloads/DL_Project/SignNet_Train.csv')
test_df = pd.read_csv('C:/Users/nagen/Downloads/DL_Project/SignNet_Test.csv')

def preprocess_data(df):
    X = df.drop(columns=['label']).values
    y = df['label'].values
    X = X / 255.0
    X = X.reshape(-1, 28, 28, 1)
    y = to_categorical(y, num_classes=25)
    return X, y

X_train, y_train = preprocess_data(train_df)
X_test, y_test = preprocess_data(test_df)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def SignNet():
    model = Sequential()
    
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.6))
    model.add(Dense(25, activation='softmax'))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

model = SignNet()

train_dataset = datagen.flow(X_train, y_train, batch_size=64)

history = model.fit(train_dataset,
                    validation_data=(X_val, y_val),
                    epochs=25,
                    callbacks=[early_stopping, reduce_lr])


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"SignNet Test accuracy: {test_acc:.4f}")

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("Classification Report of SignNet Model: ")
print(classification_report(y_true, y_pred_classes))

In [ ]:
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(25), yticklabels=np.arange(25))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix of SignNet Model')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
X_train_resized = resize(X_train, (32, 32))
X_val_resized = resize(X_val, (32, 32))
X_test_resized = resize(X_test, (32, 32))

In [ ]:
vgg_model = VGG16(weights=None, input_shape=(32, 32, 1), classes=25)
vgg_model.compile(optimizer='adam', loss=categorical_crossentropy, metrics=['accuracy'])

history_vgg = vgg_model.fit(datagen.flow(X_train_resized, y_train, batch_size=64),
                            validation_data=(X_val_resized, y_val),
                            epochs=5, callbacks=[early_stopping, reduce_lr])

In [ ]:
vgg_test_loss, vgg_test_acc = vgg_model.evaluate(X_test_resized, y_test)
print(f"VGG16 Test accuracy: {vgg_test_acc:.4f}")

y_pred = vgg_model.predict(X_test_resized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("Classification Report of VGG16 Model: ")
print(classification_report(y_true, y_pred_classes))

In [ ]:
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(25), yticklabels=np.arange(25))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix of VGG16 Model')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_vgg.history['accuracy'], label='Train Accuracy')
plt.plot(history_vgg.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history_vgg.history['loss'], label='Train Loss')
plt.plot(history_vgg.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
resnet_model = ResNet50(weights=None, input_shape=(32, 32, 1), classes=25)
resnet_model.compile(optimizer='adam', loss=categorical_crossentropy, metrics=['accuracy'])

history_resnet = resnet_model.fit(datagen.flow(X_train_resized, y_train, batch_size=64),
                                  validation_data=(X_val_resized, y_val),
                                  epochs=5, callbacks=[early_stopping, reduce_lr])

In [ ]:
resnet_test_loss, resnet_test_acc = resnet_model.evaluate(X_test_resized, y_test)
print(f"ResNet50 Test accuracy: {resnet_test_acc:.4f}")

y_pred = resnet_model.predict(X_test_resized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("Classification Report of ResNet50 Model: ")
print(classification_report(y_true, y_pred_classes))

In [ ]:
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(25), yticklabels=np.arange(25))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix of ResNet50 Model')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_resnet.history['accuracy'], label='Train Accuracy')
plt.plot(history_resnet.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history_resnet.history['loss'], label='Train Loss')
plt.plot(history_resnet.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()